In [1]:
import sys
import os

import pandas as pd
# import polars as pl

from matplotlib import pyplot as plt
%matplotlib inline  

sys.path.append('/Users/mihail/Documents/repos/analyzer')
print(sys.version)

from analyzer.preprocessing import (
    get_var_cutoffs, get_all_vars_cutoffs, binarize_series, BinningParams,
    preprocess_df, MapDictSingleVar, MapDictMultiVars, BinningParamsMultiVars
)

3.13.2 (main, Feb  4 2025, 14:51:09) [Clang 16.0.0 (clang-1600.0.26.6)]


# Подготовка
Загрузим датафрейм, на котором продемонстрируем функционал.

In [2]:
df = pd.read_csv('/Users/mihail/Documents/repos/analyzer/tests/Titanic-Dataset.csv')
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [3]:
df.shape

(891, 12)

In [4]:
df['Survived'].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

# Функционал

## get_var_cutoffs
Функция возвращает точки бининга для одной переменной (только числовой!).

In [5]:
?get_var_cutoffs

Signature:
get_var_cutoffs(
    variable: ~Series,
    target: ~Series = None,
    bin_params: analyzer.preprocessing.binning.params.BinningParams = BinningParams(min_prc=15.0, rnd=None, cutoffs=None),
    _validate_target: bool = True,
    _var_name: str = '',
    _raise_not_numeric: bool = True,
) -> list
Docstring:
Расчет точек бинаризации числовой переменной.
Args:
    variable:           переменная для бинаризации
    target:             таргет для бинаризации, опционален
    bin_params:         параметры для бининга
    _validate_target:   проверка таргета на бинарность.
    _var_name:          название переменной, вспомогательный параметр,
        помогает отслеживать выполнение функции на конкретной переменной (будет принтиться ее нейминг в логе)
    _raise_not_numeric: поднимать исключение если переменная не числовая, скрытый параметр

Returns:
    list:               числа (точки бинаризации) + спец значения (MIN, MAX)
File:      ~/Documents/repos/analyzer/analyzer/preprocess

In [6]:
# Бининг без оглядки на таргет
cutooffs = get_var_cutoffs(df['Age'])
cutooffs

['-inf', 18, 24, 30, 39, 'inf']

In [7]:
# Бининг с оглядкой на таргет
cutooffs = get_var_cutoffs(df['Age'], df['Survived'])
cutooffs

['-inf', 18.5, 26.5, 36.25, 'inf']

In [8]:
# Бининг с измененным размером минимального бакета
cutooffs = get_var_cutoffs(df['Age'], df['Survived'], bin_params=BinningParams(min_prc=25))
cutooffs

['-inf', 24.25, 'inf']

In [9]:
cutooffs = get_var_cutoffs(df['Sex'])

Exception: Вы хотите применить алгоритм категоризации к нечисловой переменной.

## get_all_vars_cutoffs

Функция для получения точек бинаризации по множеству переменных (колонок).

In [ ]:
?get_all_vars_cutoffs

In [ ]:
get_all_vars_cutoffs(df)

## binarize_series
Функция бинаризации Series.

In [ ]:
?binarize_series

In [ ]:
bin_ser = binarize_series(df['Age'])
bin_ser.head()

In [10]:
bin_ser = binarize_series(df['Age'], df['Survived'])
bin_ser.head()

0     (18.5; 26.5]
1          > 36.25
2     (18.5; 26.5]
3    (26.5; 36.25]
4    (26.5; 36.25]
Name: Age, dtype: category
Categories (5, object): ['__MISSING__' < '<= 18.5' < '(18.5; 26.5]' < '(26.5; 36.25]' < '> 36.25']

In [11]:
bin_ser = binarize_series(df['Age'], df['Survived'], bin_params=BinningParams(min_prc=25))
bin_ser.head()

0    <= 24.25
1     > 24.25
2     > 24.25
3     > 24.25
4     > 24.25
Name: Age, dtype: category
Categories (3, object): ['__MISSING__' < '<= 24.25' < '> 24.25']

In [12]:
bin_ser = binarize_series(df['Age'], df['Survived'], bin_params=BinningParams(cutoffs=[10,20,40]))
bin_ser.head()

0    (20; 40]
1    (20; 40]
2    (20; 40]
3    (20; 40]
4    (20; 40]
Name: Age, dtype: category
Categories (5, object): ['__MISSING__' < '<= 10' < '(10; 20]' < '(20; 40]' < '> 40']

## preprocess_df
Функция препроцессинга датафрейма - замена пустых значений, бинаризация, мэппинг значений.

In [13]:
?preprocess_df

Signature:
preprocess_df(
    df: ~DataFrame,
    process_vars: List[str] = None,
    ignore_vars: List[str] = None,
    target_name: str = None,
    binning: Union[Dict[str, Union[analyzer.preprocessing.binning.params.BinningParams, bool]], bool, analyzer.preprocessing.binning.params.BinningParams, List[str]] = True,
    map_values: Dict[~Variable, Dict[~OldValue, ~NewValue]] = None,
    drop_not_processed: bool = False,
    _validate_target: bool = True,
    _copy: bool = True,
    _bin_by_target: bool = True,
    _tqdm: bool = True,
) -> ~DataFrame
Docstring:
Функция препроцессинга датафрейма - замена пустых значений, бинаризация, мэппинг значений.

Args:
    df:                 Датафрейм для препроцессинга
    process_vars:       Список переменных для препроцессинга, если ничего не задать, то будут все переменные,
                        кроме таргета и игнорируемых переменных
    ignore_vars:        Список игнорируемых переменных во время препроцессинга, опционально
    target_nam

In [14]:
processed_df = preprocess_df(df, target_name='Survived', binning=True)
processed_df.head()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Бинаризация переменных, 6/6 [00:00<00:00, 211.99it/s]


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,<= 183,0,> 2,"Braund, Mr. Owen Harris",male,(18.5; 26.5],> 0,<= 0,A/5 21171,<= 7.883349895477295,__MISSING__,S
1,<= 183,1,<= 1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,> 36.25,> 0,<= 0,PC 17599,> 52.277099609375,C85,C
2,<= 183,1,> 2,"Heikkinen, Miss. Laina",female,(18.5; 26.5],<= 0,<= 0,STON/O2. 3101282,(7.883349895477295; 10.481249809265137],__MISSING__,S
3,<= 183,1,<= 1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,(26.5; 36.25],> 0,<= 0,113803,> 52.277099609375,C123,S
4,<= 183,0,> 2,"Allen, Mr. William Henry",male,(26.5; 36.25],<= 0,<= 0,373450,(7.883349895477295; 10.481249809265137],__MISSING__,S
